In [21]:
import psycopg2
import config
from flask import Flask, render_template, jsonify
import pandas as pd
import datetime as dt


In [22]:
#app = Flask(__name__)


#database='project3db', user=config.username , password=config.password, host="127.0.0.1", port="5432"

def get_db_connection():
    conn = psycopg2.connect(database='project3db', user=config.username , password=config.password, host="127.0.0.1", port="5432")
    return conn

try:
    conn = psycopg2.connect(database='project3db', user=config.username , password=config.password, host="127.0.0.1", port="5432")
 
    if conn is not None:
        print('Connection established to PostgreSQL.')
    else:
        print('Connection not established to PostgreSQL.')
         
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connection established to PostgreSQL.


In [32]:
#@app.route('/')
#def home():
conn = get_db_connection()
cur = conn.cursor()
cur.execute('SELECT week, state, confirmed FROM covid_data;')
covid_data = cur.fetchall()
covid_df= pd.DataFrame(covid_data, columns=['week', 'state', 'confirmed'])
cur.execute('SELECT f.week, f.Count FROM flu_data as f GROUP BY f.week;')
flu_data = cur.fetchall()
flu_df = pd.DataFrame(flu_data, columns=['week', 'Count'])    
cur.close()
conn.close()
# covid_groupby = covid_df.groupby('week', as_index =False)['confirmed'].sum()
# #covid_groupby['week'] = covid_groupby['week'].dt.strftime('%Y-%m-%d')
# c_week = covid_groupby.index.tolist()
# c_confirmed = covid_groupby.values.tolist()
# flu_groupby = flu_df.groupby('week')['Count'].sum()
# f_week = flu_groupby.index.tolist()
# f_count = flu_groupby.values.tolist()
# #covid_json = jsonify(covid_groupby)
covid_df
    #return render_template('index.html', covid_flu=covid_flu.to_html())


,week,state,confirmed
0,2020-01-24,ACT,0
1,2020-01-24,QLD,2
2,2020-01-24,TAS,0
3,2020-01-24,NT,0
4,2020-01-24,VIC,3
...,...,...,...
1155,2022-10-28,NT,265
1156,2022-10-28,NSW,12440
1157,2022-10-28,ACT,919
1158,2022-10-28,VIC,10153


In [33]:
covid_groupby = covid_df.groupby('week', as_index =False).apply(lambda x: x[x['state'] == 'NSW' ]['confirmed'].sum())

In [27]:
#covid_groupby
#pd.to_datetime(covid_groupby, format='%Y%m%d')

covid_groupby = covid_df.groupby('week', as_index =False)covid_df['week'].dt.strftime("%Y-%m-%d")
covid_df.dtypes

week         datetime64[ns]
confirmed             int64
dtype: object

In [19]:
for i in range(0, len(covid_df['week'])):
    covid_df['week'][i] = covid_df['week'][i].astype(str) #strftime("%Y-%m-%d")

AttributeError: 'str' object has no attribute 'astype'

In [13]:
#covid_groupby['week'] = covid_groupby['week'].dt.strftime('%Y-%m-%d')
c_week = covid_groupby['week'].astype(str).tolist()

KeyError: 'week'

In [ ]:
@app.route('/covid')
def covid():
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('SELECT * FROM covid_data;')
    covid_data = cur.fetchall()
    covid_df= pd.DataFrame(covid_data)
    cur.execute('SELECT * FROM flu_data;')
    flu_data = cur.fetchall()
    flu_df = pd.DataFrame(flu_data)
    cur.close()
    conn.close()
    covid_groupby = covid_df.groupby('week')['confirmed'].sum()
    flu_groupby = flu_df.groupby('week')['Count'].sum()
    
    return render_template('covid.html', covid_data=covid_data)


In [ ]:
@app.route('/flu')
def flu():
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute('SELECT * FROM flu_data;')
    flu_data = cur.fetchall()
    cur.close()
    conn.close()
    return render_template('flu.html', flu_data=flu_data)


In [ ]:
if __name__ == '__main__':
   app.run()